In [ ]:
#| default_exp src.rgxlog_interpreter.tests.test_general_utils

In [ ]:
#| export
import pytest
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.utils.general_utils import get_free_var_names, position_freevar_pairs, get_input_free_var_names, get_free_var_to_relations_dict, check_properly_typed_term_list, check_properly_typed_relation, type_check_rule_free_vars, type_check_rule_free_vars_aux
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.datatypes.primitive_types import DataTypes, DataTypeMapping, Span
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.state.symbol_table import SymbolTable
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.datatypes.ast_node_types import RelationDeclaration, AddFact, RemoveFact,Rule ,Assignment, Query, IERelation, Relation, get_term_list_string

In [ ]:
#| export
# Test get_free_var_names function
def test_get_free_var_names():
    term_list = ["X", 10, "Z"]
    type_list = [DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name]
    
    result = get_free_var_names(term_list, type_list)
    assert result == {"X", "Z"}

def test_get_free_var_names_empty_input():
    term_list = []
    type_list = []

    result = get_free_var_names(term_list, type_list)
    assert result == set()

def test_get_free_var_names_mismatched_lengths():
    term_list = ["X", "y", "Z"]
    type_list = [DataTypes.free_var_name, DataTypes.integer]
    
    with pytest.raises(Exception, match="received different lengths of term_list"):
        get_free_var_names(term_list, type_list)

In [ ]:
#| export
def test_position_freevar_pairs():
    term_list = ["X", "abc", "Y", "def", "Z"]
    type_list = [DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name]

    relation = Relation("relation1",term_list, type_list)

    result = position_freevar_pairs(relation)
    expected_result = [(0, "X"), (2, "Y"), (4, "Z")]

    assert result == expected_result

def test_position_freevar_pairs_empty_input():
    term_list = []
    type_list = []

    relation = Relation("relation1",term_list, type_list)

    result = position_freevar_pairs(relation)
    assert result == []

def test_position_freevar_pairs_no_free_vars():
    term_list = ["abc", "def", "ghi"]
    type_list = [DataTypes.string, DataTypes.string, DataTypes.string]

    relation = Relation("relation1",term_list, type_list)

    result = position_freevar_pairs(relation)
    assert result == []

def test_get_input_free_var_names_for_ie_relation():
    input_term_list = ["X", "Y"]
    input_type_list = [DataTypes.free_var_name, DataTypes.integer]

    ie_relation = IERelation("relation1",input_term_list, input_type_list, [], [])

    result = get_input_free_var_names(ie_relation)
    assert result == {"X"}

def test_get_input_free_var_names_for_relation():
    term_list = ["abc", "def", "ghi"]
    type_list = [DataTypes.string, DataTypes.string, DataTypes.string]
    relation = Relation("relation1",term_list, type_list)

    result = get_input_free_var_names(relation)
    assert result == set()  

{'W': [(relation3(X) -> (W), 1)],
 'X': [(relation2(Z) -> (X), 1),
       (relation3(X) -> (W), 0),
       (relation1(X, Y), 0)],
 'Y': [(relation1(X, Y), 1)],
 'Z': [(relation2(Z) -> (X), 0)]}


In [ ]:
#| export
def test_check_properly_typed_term_list_correct():
    symbol_table = SymbolTable()

    term_list = ["x", "y", "z"]
    type_list = [DataTypes.var_name, DataTypes.var_name, DataTypes.free_var_name]
    correct_type_list = [DataTypes.integer, DataTypes.string, DataTypes.free_var_name]

    symbol_table.set_var_value_and_type("x", None, DataTypes.integer)
    symbol_table.set_var_value_and_type("y", None, DataTypes.string)

    result = check_properly_typed_term_list(term_list, type_list, correct_type_list, symbol_table)
    assert result is True

def test_check_properly_typed_term_list_incorrect():
    symbol_table = SymbolTable()

    term_list = ["x", "y", "z"]
    type_list = [DataTypes.var_name, DataTypes.var_name, DataTypes.free_var_name]
    correct_type_list = [DataTypes.string, DataTypes.integer, DataTypes.free_var_name]

    symbol_table.set_var_value_and_type("x", None, DataTypes.integer)
    symbol_table.set_var_value_and_type("y", None, DataTypes.string)

    result = check_properly_typed_term_list(term_list, type_list, correct_type_list, symbol_table)
    assert result is False


def test_check_properly_typed_term_list_mismatched_lengths():
    symbol_table = SymbolTable()

    term_list = ["x", "y", "z"]
    type_list = [DataTypes.var_name, DataTypes.var_name]
    correct_type_list = [DataTypes.integer, DataTypes.string]

    symbol_table.set_var_value_and_type("x", None, DataTypes.integer)
    symbol_table.set_var_value_and_type("y", None, DataTypes.string)

    with pytest.raises(Exception, match="the length of term_list, type_list and correct_type_list should be the same"):
        check_properly_typed_term_list(term_list, type_list, correct_type_list, symbol_table)

In [ ]:
#| export
def test_check_properly_typed_relation_relation():
    symbol_table = SymbolTable()

    symbol_table.add_relation_schema("relation1", [DataTypes.integer, DataTypes.string], is_rule=False)

    relation = Relation("relation1", ["x", "y"], [DataTypes.free_var_name, DataTypes.free_var_name])
    result = check_properly_typed_relation(relation, symbol_table)
    assert result is True

def test_check_properly_typed_relation_ie_relation():
    symbol_table = SymbolTable()

    symbol_table.add_relation_schema("ie_relation1", [DataTypes.integer, DataTypes.string], is_rule=True)

    ie_relation = IERelation("ie_relation1", ["x"], [DataTypes.free_var_name], ["y"], [DataTypes.free_var_name])
    with pytest.raises(ValueError):
        check_properly_typed_relation(ie_relation, symbol_table)

def test_check_properly_typed_relation_ie_relation_incorrect_input():
    symbol_table = SymbolTable()

    symbol_table.add_relation_schema("ie_relation1", [DataTypes.integer, DataTypes.string], is_rule=True)

    ie_relation = IERelation("ie_relation1", ["x"], [DataTypes.free_var_name], ["y"], [DataTypes.string])
    with pytest.raises(ValueError):
        check_properly_typed_relation(ie_relation, symbol_table)

def test_check_properly_typed_relation_ie_relation_incorrect_output():
    symbol_table = SymbolTable()

    symbol_table.add_relation_schema("ie_relation1", [DataTypes.integer, DataTypes.string], is_rule=True)

    with pytest.raises(Exception):
        IERelation("ie_relation1", ["x"], [DataTypes.free_var_name], ["y"], [DataTypes.free_var_name, DataTypes.integer])

def test_check_properly_typed_relation_unexpected_type():
    symbol_table = SymbolTable()

    with pytest.raises(Exception, match="unexpected relation type:"):
        check_properly_typed_relation("unexpected_relation", symbol_table)

In [ ]:
#| export
def test_type_check_rule_free_vars_ierelation():
    def doSomething(x,y):
        yield x
    rule = Rule(
        "rule2",
        [IERelation("doSomething", ["x","y"], [DataTypes.free_var_name,DataTypes.free_var_name], ["x"], [DataTypes.free_var_name])],
        ["IERelation"]
    )

    symbol_table = SymbolTable()
    symbol_table.register_ie_function(doSomething,"doSomething", [DataTypes.integer, DataTypes.string], [DataTypes.integer])
    free_var_to_type, conflicted_free_vars = type_check_rule_free_vars(rule, symbol_table)
    assert free_var_to_type == {"x": DataTypes.integer, "y": DataTypes.string}
    assert conflicted_free_vars == set()
    
def test_type_check_rule_free_vars_relation():
    rule = Rule(
        "rule1",
        [Relation("Person", ["x", "y"], [DataTypes.free_var_name, DataTypes.free_var_name])],
        ["Relation"]
    )
    symbol_table = SymbolTable()
    symbol_table.add_relation_schema("Person", [DataTypes.string, DataTypes.integer], False)
    free_var_to_type, conflicted_free_vars = type_check_rule_free_vars(rule, symbol_table)
    assert free_var_to_type == {"x": DataTypes.string, "y": DataTypes.integer}
    assert conflicted_free_vars == set()

def test_type_check_rule_free_vars_conflicted_vars():
    def doSomething(x,y):
        yield x

    rule = Rule(
        "rule3",
        [
            Relation("Person", ["x", "y"], [DataTypes.free_var_name, DataTypes.free_var_name]),
            IERelation("doSomething", ["x", "y"], [DataTypes.integer, DataTypes.string], ["x"], [DataTypes.integer])
        ],
        ["Relation", "IERelation"]
    )
    symbol_table = SymbolTable()
    symbol_table.add_relation_schema("Person", [DataTypes.string, DataTypes.integer], False)
    symbol_table.register_ie_function(doSomething, "doSomething", [DataTypes.string, DataTypes.integer], [DataTypes.string])
    free_var_to_type, conflicted_free_vars = type_check_rule_free_vars(rule, symbol_table)
    assert free_var_to_type == {"x": DataTypes.string, "y": DataTypes.integer}
    assert conflicted_free_vars == set()